<a href="https://colab.research.google.com/github/Srijita264/HAR-using-image-classification/blob/main/VGG_16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os
from keras.layers import Dense, Flatten, Conv2D, Activation, Dropout
from keras import backend as K
import keras
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import MaxPool2D
from google.colab.patches import cv2_imshow
import pandas as pd
from keras.utils import to_categorical
from tensorflow.keras import layers, models
import pandas as pd

In [ ]:
y_train = pd.read_csv('/content/drive/MyDrive/y_train.csv', header=None)
y_test = pd.read_csv('/content/drive/MyDrive/y_test.csv', header=None)

# Converting labels to string
y_train[0] = y_train[0].astype(str)
y_test[0] = y_test[0].astype(str)

# Converting labels to one-hot encoding
y_train_one_hot = to_categorical(y_train[0].astype(int))
y_test_one_hot = to_categorical(y_test[0].astype(int))

# Defining the image size and batch size
img_size = (128, 128)
batch_size = 32

# Creating data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({
        'filename': ['/content/drive/MyDrive/GAF_DIFF_TRAIN/{}.png'.format(i) for i in range(1, len(y_train) + 1)],
        'label': y_train[0]
    }),
    x_col='filename',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Assuming it's a classification task
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    pd.DataFrame({
        'filename': ['/content/drive/MyDrive/GAF_DIFF_TRAIN/{}.png'.format(i) for i in range(1, len(y_train) + 1)],
        'label': y_train[0]
    }),
    x_col='filename',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Creating data generator for testing
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    pd.DataFrame({
        'filename': ['/content/drive/MyDrive/GAF_DIFF_TEST/{}.png'.format(i) for i in range(1, len(y_test) + 1)],
        'label': y_test[0]
    }),
    x_col='filename',
    y_col='label',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Conv2D(128, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2, padding='valid'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("model_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')


In [ ]:
history = model.fit(train_generator,
                    epochs=20,
                    validation_data=validation_generator,

                    callbacks=[checkpoint, early])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
print(f'Test Accuracy: {test_acc}')
